In [1]:
import sqlite3
import pandas as pd
from IPython.core.display import HTML, Markdown

In [2]:
class Report:
    def __init__(self):
        self.md = ""

    def addTitle(self, title):
        self.md += f"# {title}\n\n"

    def addSubTitle(self, title):
        self.md += f"## {title}\n\n"

    def addParagraph(self, text):
        for line in text.splitlines():
            self.md += f"{line}\n"
        self.md += "\n"

    def addText(self, text):
        self.addParagraph(text)

    def clear(self):
        self.md = ""

    def print(self):
        display(Markdown(self.md))

report = Report()

In [3]:
report.clear()

con = sqlite3.connect("data/calendario_corsi.sqlite")
# Corsi le cui edizioni devono essere visualizzate
corsi = ['CyberSec', 'CryptoWithBlocks', 'LabMat2', 'PhyComp', 'RelAx', 'RicOp']# , 'EduRobot', 'PassInf']

for c in corsi:
    sql = "SELECT idEdizione, denominazione, descrizione, idClasse FROM Edizione INNER JOIN Corso USING (idCorso) WHERE Corso.idCorso='" + c + "'"
    edizioni = pd.read_sql_query(sql, con)
    for i in range(len(edizioni)):
        edizione = edizioni.iloc[i]
        # Titolo
        report.addTitle(edizione["idEdizione"] + " - " + edizione["denominazione"])
        # Studenti
        report.addText(f"**Studenti**: {edizione["idClasse"]}")
        # Formatore
        sql = f"SELECT idPersonale FROM Formatore WHERE idEdizione = '{edizione["idEdizione"]}' AND ruolo = 'Formatore'"
        formatore = pd.read_sql_query(sql, con)["idPersonale"][0]
        report.addText(f"**Formatore**: *{formatore}*")
        # Tutor
        sql = f"SELECT idPersonale FROM Formatore WHERE idEdizione = '{edizione["idEdizione"]}' AND ruolo = 'Tutor'"
        tutor = pd.read_sql_query(sql, con)["idPersonale"][0]
        report.addText(f"**Tutor**: *{tutor}*")
        # Descriione
        report.addParagraph(f"**Descrizione**: {edizione["descrizione"]}")
        # Lezioni
        report.addText(f"**Calendario**")
        sql = f"""SELECT
                    giorno,
                    L.data as dataISO,
                    substr(L.data, 9, 2) || '/' || substr(L.data, 6, 2) || '/' || substr(L.data, 1, 4) as data,
                    L.idFasciaOraria,
                    L.idLezione
                FROM
                    Edizione E INNER JOIN 
                    Lezione L ON E.idEdizione=L.idEdizione INNER JOIN
                    Giorno G ON cast (strftime('%w', L.data) as integer)=G.idGiorno
                WHERE L.idEdizione = '{edizione["idEdizione"]}'
                ORDER BY L.data, L.idFasciaOraria"""
        lezioni = pd.read_sql_query(sql, con)
        for j in range(len(lezioni)):
            lezione = lezioni.iloc[j]
            report.addText(f"{lezione["idLezione"]}. {lezione["giorno"]} {lezione["data"]} ora n. {lezione["idFasciaOraria"]}")

# Orario formatori
report.addTitle("Orario del personale")

for p in ['Lorenzoni', 'Massi', 'Pomili', 'Sciamanna', 'Taccari']:
    report.addSubTitle(p)
    sql = f"""SELECT
                rank() OVER (ORDER BY L.data, L.idFasciaOraria) AS progressivo,
            	E.idClasse,
            	giorno,
            	L.data AS dataISO,
            	L.idFasciaOraria,
            	E.idCorso ,
            	L.idLezione,
            	F.ruolo,
            	substr(L.data, 9, 2) || '/' || substr(L.data, 6, 2) || '/' || substr(L.data, 1, 4) as data
            FROM
            	Formatore F INNER JOIN
            	Personale P ON F.idPersonale=P.idPersonale INNER JOIN
            	Edizione E ON F.idEdizione=E.idEdizione INNER JOIN
            	Lezione L ON E.idEdizione=L.idEdizione INNER JOIN
            	Giorno G ON cast (strftime('%w', L.data) as integer)=G.idGiorno
            WHERE
            	P.idPersonale = '{p}'
            ORDER BY
            	L.data,
            	L.idFasciaOraria"""
    lezioni = pd.read_sql_query(sql, con)
    for j in range(len(lezioni)):
        lezione = lezioni.iloc[j]
        report.addText(f"""{lezione["progressivo"]}. {lezione["giorno"]} {lezione["data"]} ora n. {lezione["idFasciaOraria"]} in {lezione["idClasse"]} come {lezione["ruolo"]} del corso {lezione["idCorso"]} lez. n. {lezione["idLezione"]}""")
    

con.close()
report.print()

# CyberSec3B - CYBER SICUREZZA 1

**Studenti**: S3B

**Formatore**: *Taccari*

**Tutor**: *Sciamanna*

**Descrizione**: Il percorso avvicina le studentesse e gli studenti alla sicurezza informatica tramite attività di gioco (challenge). È propedeutico alla partecipazione alle iniziative sulla sicurezza informatica proposte dalla rete CyberHighSchools (OliCyber, CyberChallenge, CyberTrials). Il laborartorio propone l'uso di un ecosistema hardware (Rasperry Pi 5) e software (sistema operativo GNU/Linux, interprete Python, strumenti per l'analisi del traffico di rete, strumenti per l'analisi dei file eseguibili) a sorgente aperto con il quale scoprire giocando le problematiche di sicurezza, sviluppare una coscienza critica sull'argomento e promuovere comportamenti orientati ad una protezione dei dati personali.

**Calendario**

1. Mer 29/01/2025 ora n. 5

2. Ven 31/01/2025 ora n. 6

3. Mer 05/02/2025 ora n. 5

4. Ven 07/02/2025 ora n. 6

5. Mer 12/02/2025 ora n. 5

6. Mer 19/02/2025 ora n. 5

7. Ven 21/02/2025 ora n. 6

8. Mer 05/03/2025 ora n. 5

9. Ven 07/03/2025 ora n. 6

10. Mer 12/03/2025 ora n. 5

11. Ven 14/03/2025 ora n. 6

12. Mer 19/03/2025 ora n. 5

13. Ven 21/03/2025 ora n. 6

14. Mer 26/03/2025 ora n. 5

15. Ven 28/03/2025 ora n. 6

16. Mer 02/04/2025 ora n. 5

17. Ven 04/04/2025 ora n. 6

18. Mer 09/04/2025 ora n. 5

19. Ven 11/04/2025 ora n. 6

20. Mer 16/04/2025 ora n. 5

# CyberSec4B - CYBER SICUREZZA 2

**Studenti**: S4B

**Formatore**: *Taccari*

**Tutor**: *Sciamanna*

**Descrizione**: Il percorso avvicina le studentesse e gli studenti alla sicurezza informatica tramite attività di gioco (challenge). È propedeutico alla partecipazione alle iniziative sulla sicurezza informatica proposte dalla rete CyberHighSchools (OliCyber, CyberChallenge, CyberTrials). Il laborartorio propone l'uso di un ecosistema hardware (Rasperry Pi 5) e software (sistema operativo GNU/Linux, interprete Python, strumenti per l'analisi del traffico di rete, strumenti per l'analisi dei file eseguibili) a sorgente aperto con il quale scoprire giocando le problematiche di sicurezza, sviluppare una coscienza critica sull'argomento e promuovere comportamenti orientati ad una protezione dei dati personali.

**Calendario**

1. Mer 29/01/2025 ora n. 6

2. Ven 31/01/2025 ora n. 4

3. Mer 05/02/2025 ora n. 6

4. Ven 07/02/2025 ora n. 4

5. Mer 12/02/2025 ora n. 6

6. Mer 19/02/2025 ora n. 6

7. Ven 21/02/2025 ora n. 4

8. Mer 26/02/2025 ora n. 6

9. Ven 28/02/2025 ora n. 4

10. Mer 05/03/2025 ora n. 6

11. Ven 07/03/2025 ora n. 4

12. Mer 12/03/2025 ora n. 6

13. Ven 14/03/2025 ora n. 4

14. Mer 19/03/2025 ora n. 6

15. Ven 21/03/2025 ora n. 4

16. Mer 26/03/2025 ora n. 6

17. Ven 28/03/2025 ora n. 4

18. Mer 02/04/2025 ora n. 6

19. Ven 04/04/2025 ora n. 4

20. Mer 09/04/2025 ora n. 6

# Crypto1B - CRITTOGRAFIA CON BLOCCHI 1

**Studenti**: S1B

**Formatore**: *Taccari*

**Tutor**: *Sciamanna*

**Descrizione**: Il corso è un'introduzione alle "grandi idee" della crittografia. L'approccio didattico procede "per scoperta" con attività che si susseguono con una successione di crittosistemi (dai classici ai più moderni). Lo studente può sperimentare caratteristiche di ciascun crittosistema, realizzado possibili procedure di attacco e valutandone i limiti. Dalle debolezze di un sistema lo studente dovrebbe avere necessità di scoprire il crittosistema successivo, che avrà superato i limiti del sistema precedente ma che potrà presentare altri aspetti limitanti. L'impianto generale del corso si basa su M. Lodi - M. Sbaraglia - S. Martini, "PROGRAMMARE PER IMPARARE LA CRITTOGRAFIA AL LICEO MATEMATICO". I crittosistemi, la loro analisi ed le possibili procedure di attacco sono realizzate in Snap! (un linguaggio di programmazione visuale a blocchi) da Michael Lodi (Università di Bologna).

**Calendario**

1. Mer 05/02/2025 ora n. 1

2. Sab 08/02/2025 ora n. 4

3. Mer 12/02/2025 ora n. 1

4. Sab 15/02/2025 ora n. 4

5. Mer 19/02/2025 ora n. 1

6. Sab 22/02/2025 ora n. 4

7. Mer 26/02/2025 ora n. 1

8. Sab 01/03/2025 ora n. 4

9. Mer 05/03/2025 ora n. 1

10. Sab 08/03/2025 ora n. 4

# Crypto2B - CRITTOGRAFIA CON LINGUAGGIO A BLOCCHI

**Studenti**: S2B

**Formatore**: *Massi*

**Tutor**: *Pomili*

**Descrizione**: Il corso è un'introduzione alle "grandi idee" della crittografia. L'approccio didattico procede "per scoperta" con attività che si susseguono con una successione di crittosistemi (dai classici ai più moderni). Lo studente può sperimentare caratteristiche di ciascun crittosistema, realizzado possibili procedure di attacco e valutandone i limiti. Dalle debolezze di un sistema lo studente dovrebbe avere necessità di scoprire il crittosistema successivo, che avrà superato i limiti del sistema precedente ma che potrà presentare altri aspetti limitanti. L'impianto generale del corso si basa su M. Lodi - M. Sbaraglia - S. Martini, "PROGRAMMARE PER IMPARARE LA CRITTOGRAFIA AL LICEO MATEMATICO". I crittosistemi, la loro analisi ed le possibili procedure di attacco sono realizzate in Snap! (un linguaggio di programmazione visuale a blocchi) da Michael Lodi (Università di Bologna).

**Calendario**

1. Sab 01/02/2025 ora n. 1

2. Mer 05/02/2025 ora n. 4

3. Sab 08/02/2025 ora n. 1

4. Mer 12/02/2025 ora n. 4

5. Sab 15/02/2025 ora n. 1

6. Mer 19/02/2025 ora n. 4

7. Sab 22/02/2025 ora n. 1

8. Sab 01/03/2025 ora n. 1

9. Mer 05/03/2025 ora n. 4

10. Sab 08/03/2025 ora n. 1

# Crypto4G - CRITTOGRAFIA CON BLOCCHI 2

**Studenti**: S4G

**Formatore**: *Taccari*

**Tutor**: *Lorenzoni*

**Descrizione**: Il corso è un'introduzione alle "grandi idee" della crittografia. L'approccio didattico procede "per scoperta" con attività che si susseguono con una successione di crittosistemi (dai classici ai più moderni). Lo studente può sperimentare caratteristiche di ciascun crittosistema, realizzado possibili procedure di attacco e valutandone i limiti. Dalle debolezze di un sistema lo studente dovrebbe avere necessità di scoprire il crittosistema successivo, che avrà superato i limiti del sistema precedente ma che potrà presentare altri aspetti limitanti. L'impianto generale del corso si basa su M. Lodi - M. Sbaraglia - S. Martini, "PROGRAMMARE PER IMPARARE LA CRITTOGRAFIA AL LICEO MATEMATICO". I crittosistemi, la loro analisi ed le possibili procedure di attacco sono realizzate in Snap! (un linguaggio di programmazione visuale a blocchi) da Michael Lodi (Università di Bologna).

**Calendario**

1. Mar 04/02/2025 ora n. 2

2. Mar 11/02/2025 ora n. 2

3. Mar 18/02/2025 ora n. 2

4. Mar 25/02/2025 ora n. 2

5. Mar 04/03/2025 ora n. 2

6. Mar 11/03/2025 ora n. 2

7. Mar 18/03/2025 ora n. 2

8. Mar 25/03/2025 ora n. 2

9. Mar 01/04/2025 ora n. 2

10. Mar 08/04/2025 ora n. 2

# LM2 - LABORATORIO DI MATEMATICA 2

**Studenti**: LicMat

**Formatore**: *Massi*

**Tutor**: *Marrucchiello*

**Descrizione**: Il corso mira a sviluppare il pensiero ricorsivo attraverso lo sviluppo dell'aritmetica. Tra induzione e ricorsione saranno definiti l'insieme dei numeri naturali e, con le sole funzioni primitive di riconoscimento dello zero, successore e predecessore, saranno definiti alcuni operatori di confronto e operazioni di addizione, sottrazione, moltiplicazione e divisione intera. Lo stesso modo di ragionare su problemi numerici può essere applicato alla risoluzione di tutti problemi che l'informatica può risolvere. Sono questi ragionamenti il vero oggetto del corso.

**Calendario**

1. Ven 31/01/2025 ora n. 5

2. Ven 31/01/2025 ora n. 6

3. Ven 07/02/2025 ora n. 5

4. Ven 07/02/2025 ora n. 6

5. Ven 14/02/2025 ora n. 5

6. Ven 14/02/2025 ora n. 6

7. Ven 21/02/2025 ora n. 5

8. Ven 21/02/2025 ora n. 6

9. Ven 07/03/2025 ora n. 5

10. Ven 07/03/2025 ora n. 6

# PhyComp - PHYSICAL COMPUTING

**Studenti**: E2B

**Formatore**: *Massi*

**Tutor**: *Pomili*

**Descrizione**: Il percorso vuole fornire un'introduzione al Physical computing e usando Micro:bit, Raspberry Pi e Arduino. L'uso delle schede elettroniche programmabili permette l'interazione con il mondo reale e rende visibile l'effetto dell'esecuzione del programma che lo studente andrà a realizzare. Obiettivi del corso sono la 

**Calendario**

1. Lun 10/03/2025 ora n. 2

2. Mer 12/03/2025 ora n. 4

3. Lun 17/03/2025 ora n. 2

4. Mer 19/03/2025 ora n. 4

5. Lun 24/03/2025 ora n. 2

6. Mer 26/03/2025 ora n. 4

7. Lun 31/03/2025 ora n. 2

8. Mer 02/04/2025 ora n. 4

9. Lun 07/04/2025 ora n. 2

10. Mer 09/04/2025 ora n. 4

# RelAx4E - ALGEBRA DELLE RELAZIONI 2

**Studenti**: S4E

**Formatore**: *Massi*

**Tutor**: *Pomili*

**Descrizione**: Il percorso introduce le basi della teoria delle basi di dati relazionali, un pilastro fondamentale dell'informatica intesa come disciplina che studia i metodi per estrarre informazioni in modo automatico da dati grezzi.
La presentazione degli argomenti è declinata in modo rendere evidente come l'informatica sia una disciplina scientifica e ingegneristica. Vengono esposti sia gli aspetti scientifici e formali, come quelli dell'algebra delle relazioni, sia quelli ingegneristici legati all'efficienza computazionale degli algoritmi.
I contenuti si collocano in un contesto interdisciplinare, come è proprio dell'informatica, dove la teoria si unisce alla pratica, grazie all'utilizzo di una piattaforma web per la valutazione di espressioni algebriche relazionali.
Obiettivi generali:
  -  cogliere la stretta relazione tra pensiero scientifico e sviluppo tecnologico.
  - comprendere le strutture fondamentali dei ragionamenti logico-deduttivi e padroneggiare il linguaggio logico-formale per risolvere problemi di varia natura.
  - utilizzare strumenti informatici per modellare e risolvere problemi.
  - acquisire una comprensione approfondita della disciplina informatica per orientare le proprie scelte future, sia a livello universitario che professionale.

**Calendario**

1. Mer 05/02/2025 ora n. 2

2. Mer 12/02/2025 ora n. 2

3. Mer 19/02/2025 ora n. 2

4. Mer 26/02/2025 ora n. 2

5. Mer 05/03/2025 ora n. 2

6. Mer 12/03/2025 ora n. 2

7. Mer 19/03/2025 ora n. 2

8. Mer 26/03/2025 ora n. 2

9. Mer 02/04/2025 ora n. 2

10. Mer 09/04/2025 ora n. 2

# RelAx4F - ALGEBRA DELLE RELAZIONI 4

**Studenti**: S4F

**Formatore**: *Massi*

**Tutor**: *Sciamanna*

**Descrizione**: Il percorso introduce le basi della teoria delle basi di dati relazionali, un pilastro fondamentale dell'informatica intesa come disciplina che studia i metodi per estrarre informazioni in modo automatico da dati grezzi.
La presentazione degli argomenti è declinata in modo rendere evidente come l'informatica sia una disciplina scientifica e ingegneristica. Vengono esposti sia gli aspetti scientifici e formali, come quelli dell'algebra delle relazioni, sia quelli ingegneristici legati all'efficienza computazionale degli algoritmi.
I contenuti si collocano in un contesto interdisciplinare, come è proprio dell'informatica, dove la teoria si unisce alla pratica, grazie all'utilizzo di una piattaforma web per la valutazione di espressioni algebriche relazionali.
Obiettivi generali:
  -  cogliere la stretta relazione tra pensiero scientifico e sviluppo tecnologico.
  - comprendere le strutture fondamentali dei ragionamenti logico-deduttivi e padroneggiare il linguaggio logico-formale per risolvere problemi di varia natura.
  - utilizzare strumenti informatici per modellare e risolvere problemi.
  - acquisire una comprensione approfondita della disciplina informatica per orientare le proprie scelte future, sia a livello universitario che professionale.

**Calendario**

1. Mer 11/12/2024 ora n. 4

2. Lun 16/12/2024 ora n. 1

3. Mer 18/12/2024 ora n. 4

4. Mer 08/01/2025 ora n. 4

5. Lun 13/01/2025 ora n. 1

6. Mer 15/01/2025 ora n. 4

7. Lun 20/01/2025 ora n. 1

8. Mer 22/01/2025 ora n. 4

9. Lun 27/01/2025 ora n. 1

10. Mer 29/01/2025 ora n. 4

# RelAx5LSinf - ALGEBRA DELLE RELAZIONI 1

**Studenti**: S5B+S5D

**Formatore**: *Massi*

**Tutor**: *Pomili*

**Descrizione**: Il percorso introduce le basi della teoria delle basi di dati relazionali, un pilastro fondamentale dell'informatica intesa come disciplina che studia i metodi per estrarre informazioni in modo automatico da dati grezzi.
La presentazione degli argomenti è declinata in modo rendere evidente come l'informatica sia una disciplina scientifica e ingegneristica. Vengono esposti sia gli aspetti scientifici e formali, come quelli dell'algebra delle relazioni, sia quelli ingegneristici legati all'efficienza computazionale degli algoritmi.
I contenuti si collocano in un contesto interdisciplinare, come è proprio dell'informatica, dove la teoria si unisce alla pratica, grazie all'utilizzo di una piattaforma web per la valutazione di espressioni algebriche relazionali.
Obiettivi generali:
  -  cogliere la stretta relazione tra pensiero scientifico e sviluppo tecnologico.
  - comprendere le strutture fondamentali dei ragionamenti logico-deduttivi e padroneggiare il linguaggio logico-formale per risolvere problemi di varia natura.
  - utilizzare strumenti informatici per modellare e risolvere problemi.
  - acquisire una comprensione approfondita della disciplina informatica per orientare le proprie scelte future, sia a livello universitario che professionale.

**Calendario**

1. Ven 13/12/2024 ora n. 5

2. Ven 20/12/2024 ora n. 5

3. Ven 10/01/2025 ora n. 5

4. Ven 17/01/2025 ora n. 5

5. Ven 24/01/2025 ora n. 5

6. Ven 14/03/2025 ora n. 5

7. Ven 21/03/2025 ora n. 5

8. Ven 28/03/2025 ora n. 5

9. Ven 04/04/2025 ora n. 5

10. Ven 11/04/2025 ora n. 5

# RelAx5E - ALGEBRA DELLE RELAZIONI 3

**Studenti**: S5E

**Formatore**: *Massi*

**Tutor**: *Sciamanna*

**Descrizione**: Il percorso introduce le basi della teoria delle basi di dati relazionali, un pilastro fondamentale dell'informatica intesa come disciplina che studia i metodi per estrarre informazioni in modo automatico da dati grezzi.
La presentazione degli argomenti è declinata in modo rendere evidente come l'informatica sia una disciplina scientifica e ingegneristica. Vengono esposti sia gli aspetti scientifici e formali, come quelli dell'algebra delle relazioni, sia quelli ingegneristici legati all'efficienza computazionale degli algoritmi.
I contenuti si collocano in un contesto interdisciplinare, come è proprio dell'informatica, dove la teoria si unisce alla pratica, grazie all'utilizzo di una piattaforma web per la valutazione di espressioni algebriche relazionali.
Obiettivi generali:
  -  cogliere la stretta relazione tra pensiero scientifico e sviluppo tecnologico.
  - comprendere le strutture fondamentali dei ragionamenti logico-deduttivi e padroneggiare il linguaggio logico-formale per risolvere problemi di varia natura.
  - utilizzare strumenti informatici per modellare e risolvere problemi.
  - acquisire una comprensione approfondita della disciplina informatica per orientare le proprie scelte future, sia a livello universitario che professionale.

**Calendario**

1. Mer 11/12/2024 ora n. 3

2. Lun 16/12/2024 ora n. 2

3. Mer 18/12/2024 ora n. 3

4. Mer 08/01/2025 ora n. 3

5. Lun 13/01/2025 ora n. 2

6. Mer 15/01/2025 ora n. 3

7. Lun 20/01/2025 ora n. 2

8. Mer 22/01/2025 ora n. 3

9. Lun 27/01/2025 ora n. 2

10. Mer 29/01/2025 ora n. 3

# RicOp5A - RICERCA OPERATIVA

**Studenti**: E5ASIA

**Formatore**: *Marinelli*

**Tutor**: *Massi*

**Descrizione**: Il laboratorio introduce, per mezzo di attività laboratoriali, le problematiche affrontate della Ricerca Operativa, una disciplina che si avvale di metodi scientifici, matematici e informatici per risolvere problemi complessi che si presentano in svariati contesti, dalla logistica all'economia, passando per l'ingegneria e la medicina.

**Calendario**

1. Lun 03/02/2025 ora n. 1

2. Lun 03/02/2025 ora n. 2

3. Lun 10/02/2025 ora n. 1

4. Lun 10/02/2025 ora n. 2

5. Lun 17/02/2025 ora n. 1

6. Lun 17/02/2025 ora n. 2

7. Lun 24/02/2025 ora n. 1

8. Lun 24/02/2025 ora n. 2

9. Lun 03/03/2025 ora n. 1

10. Lun 03/03/2025 ora n. 2

# Orario del personale

## Lorenzoni

1. Mar 04/02/2025 ora n. 2 in S4G come Tutor del corso CryptoWithBlocks lez. n. 1

2. Mar 11/02/2025 ora n. 2 in S4G come Tutor del corso CryptoWithBlocks lez. n. 2

3. Mar 18/02/2025 ora n. 2 in S4G come Tutor del corso CryptoWithBlocks lez. n. 3

4. Mar 25/02/2025 ora n. 2 in S4G come Tutor del corso CryptoWithBlocks lez. n. 4

5. Mar 04/03/2025 ora n. 2 in S4G come Tutor del corso CryptoWithBlocks lez. n. 5

6. Mar 11/03/2025 ora n. 2 in S4G come Tutor del corso CryptoWithBlocks lez. n. 6

7. Mar 18/03/2025 ora n. 2 in S4G come Tutor del corso CryptoWithBlocks lez. n. 7

8. Mar 25/03/2025 ora n. 2 in S4G come Tutor del corso CryptoWithBlocks lez. n. 8

9. Mar 01/04/2025 ora n. 2 in S4G come Tutor del corso CryptoWithBlocks lez. n. 9

10. Mar 08/04/2025 ora n. 2 in S4G come Tutor del corso CryptoWithBlocks lez. n. 10

## Massi

1. Mer 11/12/2024 ora n. 3 in S5E come Formatore del corso RelAx lez. n. 1

2. Mer 11/12/2024 ora n. 4 in S4F come Formatore del corso RelAx lez. n. 1

3. Ven 13/12/2024 ora n. 5 in S5B+S5D come Formatore del corso RelAx lez. n. 1

4. Lun 16/12/2024 ora n. 1 in S4F come Formatore del corso RelAx lez. n. 2

5. Lun 16/12/2024 ora n. 2 in S5E come Formatore del corso RelAx lez. n. 2

6. Mer 18/12/2024 ora n. 3 in S5E come Formatore del corso RelAx lez. n. 3

7. Mer 18/12/2024 ora n. 4 in S4F come Formatore del corso RelAx lez. n. 3

8. Ven 20/12/2024 ora n. 5 in S5B+S5D come Formatore del corso RelAx lez. n. 2

9. Mer 08/01/2025 ora n. 3 in S5E come Formatore del corso RelAx lez. n. 4

10. Mer 08/01/2025 ora n. 4 in S4F come Formatore del corso RelAx lez. n. 4

11. Ven 10/01/2025 ora n. 5 in S5B+S5D come Formatore del corso RelAx lez. n. 3

12. Lun 13/01/2025 ora n. 1 in S4F come Formatore del corso RelAx lez. n. 5

13. Lun 13/01/2025 ora n. 2 in S5E come Formatore del corso RelAx lez. n. 5

14. Mer 15/01/2025 ora n. 3 in S5E come Formatore del corso RelAx lez. n. 6

15. Mer 15/01/2025 ora n. 4 in S4F come Formatore del corso RelAx lez. n. 6

16. Ven 17/01/2025 ora n. 5 in S5B+S5D come Formatore del corso RelAx lez. n. 4

17. Lun 20/01/2025 ora n. 1 in S4F come Formatore del corso RelAx lez. n. 7

18. Lun 20/01/2025 ora n. 2 in S5E come Formatore del corso RelAx lez. n. 7

19. Mer 22/01/2025 ora n. 3 in S5E come Formatore del corso RelAx lez. n. 8

20. Mer 22/01/2025 ora n. 4 in S4F come Formatore del corso RelAx lez. n. 8

21. Ven 24/01/2025 ora n. 5 in S5B+S5D come Formatore del corso RelAx lez. n. 5

22. Lun 27/01/2025 ora n. 1 in S4F come Formatore del corso RelAx lez. n. 9

23. Lun 27/01/2025 ora n. 2 in S5E come Formatore del corso RelAx lez. n. 9

24. Mer 29/01/2025 ora n. 3 in S5E come Formatore del corso RelAx lez. n. 10

25. Mer 29/01/2025 ora n. 4 in S4F come Formatore del corso RelAx lez. n. 10

26. Ven 31/01/2025 ora n. 5 in LicMat come Formatore del corso LabMat2 lez. n. 1

27. Ven 31/01/2025 ora n. 6 in LicMat come Formatore del corso LabMat2 lez. n. 2

28. Sab 01/02/2025 ora n. 1 in S2B come Formatore del corso CryptoWithBlocks lez. n. 1

29. Lun 03/02/2025 ora n. 1 in E5ASIA come Tutor del corso RicOp lez. n. 1

30. Lun 03/02/2025 ora n. 2 in E5ASIA come Tutor del corso RicOp lez. n. 2

31. Mer 05/02/2025 ora n. 2 in S4E come Formatore del corso RelAx lez. n. 1

32. Mer 05/02/2025 ora n. 4 in S2B come Formatore del corso CryptoWithBlocks lez. n. 2

33. Ven 07/02/2025 ora n. 5 in LicMat come Formatore del corso LabMat2 lez. n. 3

34. Ven 07/02/2025 ora n. 6 in LicMat come Formatore del corso LabMat2 lez. n. 4

35. Sab 08/02/2025 ora n. 1 in S2B come Formatore del corso CryptoWithBlocks lez. n. 3

36. Lun 10/02/2025 ora n. 1 in E5ASIA come Tutor del corso RicOp lez. n. 3

37. Lun 10/02/2025 ora n. 2 in E5ASIA come Tutor del corso RicOp lez. n. 4

38. Mer 12/02/2025 ora n. 2 in S4E come Formatore del corso RelAx lez. n. 2

39. Mer 12/02/2025 ora n. 4 in S2B come Formatore del corso CryptoWithBlocks lez. n. 4

40. Ven 14/02/2025 ora n. 5 in LicMat come Formatore del corso LabMat2 lez. n. 5

41. Ven 14/02/2025 ora n. 6 in LicMat come Formatore del corso LabMat2 lez. n. 6

42. Sab 15/02/2025 ora n. 1 in S2B come Formatore del corso CryptoWithBlocks lez. n. 5

43. Lun 17/02/2025 ora n. 1 in E5ASIA come Tutor del corso RicOp lez. n. 5

44. Lun 17/02/2025 ora n. 2 in E5ASIA come Tutor del corso RicOp lez. n. 6

45. Mer 19/02/2025 ora n. 2 in S4E come Formatore del corso RelAx lez. n. 3

46. Mer 19/02/2025 ora n. 4 in S2B come Formatore del corso CryptoWithBlocks lez. n. 6

47. Ven 21/02/2025 ora n. 5 in LicMat come Formatore del corso LabMat2 lez. n. 7

48. Ven 21/02/2025 ora n. 6 in LicMat come Formatore del corso LabMat2 lez. n. 8

49. Sab 22/02/2025 ora n. 1 in S2B come Formatore del corso CryptoWithBlocks lez. n. 7

50. Lun 24/02/2025 ora n. 1 in E5ASIA come Tutor del corso RicOp lez. n. 7

51. Lun 24/02/2025 ora n. 2 in E5ASIA come Tutor del corso RicOp lez. n. 8

52. Mer 26/02/2025 ora n. 2 in S4E come Formatore del corso RelAx lez. n. 4

53. Sab 01/03/2025 ora n. 1 in S2B come Formatore del corso CryptoWithBlocks lez. n. 8

54. Lun 03/03/2025 ora n. 1 in E5ASIA come Tutor del corso RicOp lez. n. 9

55. Lun 03/03/2025 ora n. 2 in E5ASIA come Tutor del corso RicOp lez. n. 10

56. Mer 05/03/2025 ora n. 2 in S4E come Formatore del corso RelAx lez. n. 5

57. Mer 05/03/2025 ora n. 4 in S2B come Formatore del corso CryptoWithBlocks lez. n. 9

58. Ven 07/03/2025 ora n. 5 in LicMat come Formatore del corso LabMat2 lez. n. 9

59. Ven 07/03/2025 ora n. 6 in LicMat come Formatore del corso LabMat2 lez. n. 10

60. Sab 08/03/2025 ora n. 1 in S2B come Formatore del corso CryptoWithBlocks lez. n. 10

61. Lun 10/03/2025 ora n. 2 in E2B come Formatore del corso PhyComp lez. n. 1

62. Mer 12/03/2025 ora n. 2 in S4E come Formatore del corso RelAx lez. n. 6

63. Mer 12/03/2025 ora n. 4 in E2B come Formatore del corso PhyComp lez. n. 2

64. Ven 14/03/2025 ora n. 5 in S5B+S5D come Formatore del corso RelAx lez. n. 6

65. Lun 17/03/2025 ora n. 2 in E2B come Formatore del corso PhyComp lez. n. 3

66. Mer 19/03/2025 ora n. 2 in S4E come Formatore del corso RelAx lez. n. 7

67. Mer 19/03/2025 ora n. 4 in E2B come Formatore del corso PhyComp lez. n. 4

68. Ven 21/03/2025 ora n. 5 in S5B+S5D come Formatore del corso RelAx lez. n. 7

69. Lun 24/03/2025 ora n. 2 in E2B come Formatore del corso PhyComp lez. n. 5

70. Mer 26/03/2025 ora n. 2 in S4E come Formatore del corso RelAx lez. n. 8

71. Mer 26/03/2025 ora n. 4 in E2B come Formatore del corso PhyComp lez. n. 6

72. Ven 28/03/2025 ora n. 5 in S5B+S5D come Formatore del corso RelAx lez. n. 8

73. Lun 31/03/2025 ora n. 2 in E2B come Formatore del corso PhyComp lez. n. 7

74. Mer 02/04/2025 ora n. 2 in S4E come Formatore del corso RelAx lez. n. 9

75. Mer 02/04/2025 ora n. 4 in E2B come Formatore del corso PhyComp lez. n. 8

76. Ven 04/04/2025 ora n. 5 in S5B+S5D come Formatore del corso RelAx lez. n. 9

77. Lun 07/04/2025 ora n. 2 in E2B come Formatore del corso PhyComp lez. n. 9

78. Mer 09/04/2025 ora n. 2 in S4E come Formatore del corso RelAx lez. n. 10

79. Mer 09/04/2025 ora n. 4 in E2B come Formatore del corso PhyComp lez. n. 10

80. Ven 11/04/2025 ora n. 5 in S5B+S5D come Formatore del corso RelAx lez. n. 10

## Pomili

1. Ven 13/12/2024 ora n. 5 in S5B+S5D come Tutor del corso RelAx lez. n. 1

2. Ven 20/12/2024 ora n. 5 in S5B+S5D come Tutor del corso RelAx lez. n. 2

3. Ven 10/01/2025 ora n. 5 in S5B+S5D come Tutor del corso RelAx lez. n. 3

4. Ven 17/01/2025 ora n. 5 in S5B+S5D come Tutor del corso RelAx lez. n. 4

5. Ven 24/01/2025 ora n. 5 in S5B+S5D come Tutor del corso RelAx lez. n. 5

6. Sab 01/02/2025 ora n. 1 in S2B come Tutor del corso CryptoWithBlocks lez. n. 1

7. Mer 05/02/2025 ora n. 2 in S4E come Tutor del corso RelAx lez. n. 1

8. Mer 05/02/2025 ora n. 4 in S2B come Tutor del corso CryptoWithBlocks lez. n. 2

9. Sab 08/02/2025 ora n. 1 in S2B come Tutor del corso CryptoWithBlocks lez. n. 3

10. Mer 12/02/2025 ora n. 2 in S4E come Tutor del corso RelAx lez. n. 2

11. Mer 12/02/2025 ora n. 4 in S2B come Tutor del corso CryptoWithBlocks lez. n. 4

12. Sab 15/02/2025 ora n. 1 in S2B come Tutor del corso CryptoWithBlocks lez. n. 5

13. Mer 19/02/2025 ora n. 2 in S4E come Tutor del corso RelAx lez. n. 3

14. Mer 19/02/2025 ora n. 4 in S2B come Tutor del corso CryptoWithBlocks lez. n. 6

15. Sab 22/02/2025 ora n. 1 in S2B come Tutor del corso CryptoWithBlocks lez. n. 7

16. Mer 26/02/2025 ora n. 2 in S4E come Tutor del corso RelAx lez. n. 4

17. Sab 01/03/2025 ora n. 1 in S2B come Tutor del corso CryptoWithBlocks lez. n. 8

18. Mer 05/03/2025 ora n. 2 in S4E come Tutor del corso RelAx lez. n. 5

19. Mer 05/03/2025 ora n. 4 in S2B come Tutor del corso CryptoWithBlocks lez. n. 9

20. Sab 08/03/2025 ora n. 1 in S2B come Tutor del corso CryptoWithBlocks lez. n. 10

21. Lun 10/03/2025 ora n. 2 in E2B come Tutor del corso PhyComp lez. n. 1

22. Mer 12/03/2025 ora n. 2 in S4E come Tutor del corso RelAx lez. n. 6

23. Mer 12/03/2025 ora n. 4 in E2B come Tutor del corso PhyComp lez. n. 2

24. Ven 14/03/2025 ora n. 5 in S5B+S5D come Tutor del corso RelAx lez. n. 6

25. Lun 17/03/2025 ora n. 2 in E2B come Tutor del corso PhyComp lez. n. 3

26. Mer 19/03/2025 ora n. 2 in S4E come Tutor del corso RelAx lez. n. 7

27. Mer 19/03/2025 ora n. 4 in E2B come Tutor del corso PhyComp lez. n. 4

28. Ven 21/03/2025 ora n. 5 in S5B+S5D come Tutor del corso RelAx lez. n. 7

29. Lun 24/03/2025 ora n. 2 in E2B come Tutor del corso PhyComp lez. n. 5

30. Mer 26/03/2025 ora n. 2 in S4E come Tutor del corso RelAx lez. n. 8

31. Mer 26/03/2025 ora n. 4 in E2B come Tutor del corso PhyComp lez. n. 6

32. Ven 28/03/2025 ora n. 5 in S5B+S5D come Tutor del corso RelAx lez. n. 8

33. Lun 31/03/2025 ora n. 2 in E2B come Tutor del corso PhyComp lez. n. 7

34. Mer 02/04/2025 ora n. 2 in S4E come Tutor del corso RelAx lez. n. 9

35. Mer 02/04/2025 ora n. 4 in E2B come Tutor del corso PhyComp lez. n. 8

36. Ven 04/04/2025 ora n. 5 in S5B+S5D come Tutor del corso RelAx lez. n. 9

37. Lun 07/04/2025 ora n. 2 in E2B come Tutor del corso PhyComp lez. n. 9

38. Mer 09/04/2025 ora n. 2 in S4E come Tutor del corso RelAx lez. n. 10

39. Mer 09/04/2025 ora n. 4 in E2B come Tutor del corso PhyComp lez. n. 10

40. Ven 11/04/2025 ora n. 5 in S5B+S5D come Tutor del corso RelAx lez. n. 10

## Sciamanna

1. Mer 11/12/2024 ora n. 3 in S5E come Tutor del corso RelAx lez. n. 1

2. Mer 11/12/2024 ora n. 4 in S4F come Tutor del corso RelAx lez. n. 1

3. Lun 16/12/2024 ora n. 1 in S4F come Tutor del corso RelAx lez. n. 2

4. Lun 16/12/2024 ora n. 2 in S5E come Tutor del corso RelAx lez. n. 2

5. Mer 18/12/2024 ora n. 3 in S5E come Tutor del corso RelAx lez. n. 3

6. Mer 18/12/2024 ora n. 4 in S4F come Tutor del corso RelAx lez. n. 3

7. Mer 08/01/2025 ora n. 3 in S5E come Tutor del corso RelAx lez. n. 4

8. Mer 08/01/2025 ora n. 4 in S4F come Tutor del corso RelAx lez. n. 4

9. Lun 13/01/2025 ora n. 1 in S4F come Tutor del corso RelAx lez. n. 5

10. Lun 13/01/2025 ora n. 2 in S5E come Tutor del corso RelAx lez. n. 5

11. Mer 15/01/2025 ora n. 3 in S5E come Tutor del corso RelAx lez. n. 6

12. Mer 15/01/2025 ora n. 4 in S4F come Tutor del corso RelAx lez. n. 6

13. Lun 20/01/2025 ora n. 1 in S4F come Tutor del corso RelAx lez. n. 7

14. Lun 20/01/2025 ora n. 2 in S5E come Tutor del corso RelAx lez. n. 7

15. Mer 22/01/2025 ora n. 3 in S5E come Tutor del corso RelAx lez. n. 8

16. Mer 22/01/2025 ora n. 4 in S4F come Tutor del corso RelAx lez. n. 8

17. Lun 27/01/2025 ora n. 1 in S4F come Tutor del corso RelAx lez. n. 9

18. Lun 27/01/2025 ora n. 2 in S5E come Tutor del corso RelAx lez. n. 9

19. Mer 29/01/2025 ora n. 3 in S5E come Tutor del corso RelAx lez. n. 10

20. Mer 29/01/2025 ora n. 4 in S4F come Tutor del corso RelAx lez. n. 10

21. Mer 29/01/2025 ora n. 5 in S3B come Tutor del corso CyberSec lez. n. 1

22. Mer 29/01/2025 ora n. 6 in S4B come Tutor del corso CyberSec lez. n. 1

23. Ven 31/01/2025 ora n. 4 in S4B come Tutor del corso CyberSec lez. n. 2

24. Ven 31/01/2025 ora n. 6 in S3B come Tutor del corso CyberSec lez. n. 2

25. Mer 05/02/2025 ora n. 1 in S1B come Tutor del corso CryptoWithBlocks lez. n. 1

26. Mer 05/02/2025 ora n. 5 in S3B come Tutor del corso CyberSec lez. n. 3

27. Mer 05/02/2025 ora n. 6 in S4B come Tutor del corso CyberSec lez. n. 3

28. Ven 07/02/2025 ora n. 4 in S4B come Tutor del corso CyberSec lez. n. 4

29. Ven 07/02/2025 ora n. 6 in S3B come Tutor del corso CyberSec lez. n. 4

30. Sab 08/02/2025 ora n. 4 in S1B come Tutor del corso CryptoWithBlocks lez. n. 2

31. Mer 12/02/2025 ora n. 1 in S1B come Tutor del corso CryptoWithBlocks lez. n. 3

32. Mer 12/02/2025 ora n. 5 in S3B come Tutor del corso CyberSec lez. n. 5

33. Mer 12/02/2025 ora n. 6 in S4B come Tutor del corso CyberSec lez. n. 5

34. Sab 15/02/2025 ora n. 4 in S1B come Tutor del corso CryptoWithBlocks lez. n. 4

35. Mer 19/02/2025 ora n. 1 in S1B come Tutor del corso CryptoWithBlocks lez. n. 5

36. Mer 19/02/2025 ora n. 5 in S3B come Tutor del corso CyberSec lez. n. 6

37. Mer 19/02/2025 ora n. 6 in S4B come Tutor del corso CyberSec lez. n. 6

38. Ven 21/02/2025 ora n. 4 in S4B come Tutor del corso CyberSec lez. n. 7

39. Ven 21/02/2025 ora n. 6 in S3B come Tutor del corso CyberSec lez. n. 7

40. Sab 22/02/2025 ora n. 4 in S1B come Tutor del corso CryptoWithBlocks lez. n. 6

41. Mer 26/02/2025 ora n. 1 in S1B come Tutor del corso CryptoWithBlocks lez. n. 7

42. Mer 26/02/2025 ora n. 6 in S4B come Tutor del corso CyberSec lez. n. 8

43. Ven 28/02/2025 ora n. 4 in S4B come Tutor del corso CyberSec lez. n. 9

44. Sab 01/03/2025 ora n. 4 in S1B come Tutor del corso CryptoWithBlocks lez. n. 8

45. Mer 05/03/2025 ora n. 1 in S1B come Tutor del corso CryptoWithBlocks lez. n. 9

46. Mer 05/03/2025 ora n. 5 in S3B come Tutor del corso CyberSec lez. n. 8

47. Mer 05/03/2025 ora n. 6 in S4B come Tutor del corso CyberSec lez. n. 10

48. Ven 07/03/2025 ora n. 4 in S4B come Tutor del corso CyberSec lez. n. 11

49. Ven 07/03/2025 ora n. 6 in S3B come Tutor del corso CyberSec lez. n. 9

50. Sab 08/03/2025 ora n. 4 in S1B come Tutor del corso CryptoWithBlocks lez. n. 10

51. Mer 12/03/2025 ora n. 5 in S3B come Tutor del corso CyberSec lez. n. 10

52. Mer 12/03/2025 ora n. 6 in S4B come Tutor del corso CyberSec lez. n. 12

53. Ven 14/03/2025 ora n. 4 in S4B come Tutor del corso CyberSec lez. n. 13

54. Ven 14/03/2025 ora n. 6 in S3B come Tutor del corso CyberSec lez. n. 11

55. Mer 19/03/2025 ora n. 5 in S3B come Tutor del corso CyberSec lez. n. 12

56. Mer 19/03/2025 ora n. 6 in S4B come Tutor del corso CyberSec lez. n. 14

57. Ven 21/03/2025 ora n. 4 in S4B come Tutor del corso CyberSec lez. n. 15

58. Ven 21/03/2025 ora n. 6 in S3B come Tutor del corso CyberSec lez. n. 13

59. Mer 26/03/2025 ora n. 5 in S3B come Tutor del corso CyberSec lez. n. 14

60. Mer 26/03/2025 ora n. 6 in S4B come Tutor del corso CyberSec lez. n. 16

61. Ven 28/03/2025 ora n. 4 in S4B come Tutor del corso CyberSec lez. n. 17

62. Ven 28/03/2025 ora n. 6 in S3B come Tutor del corso CyberSec lez. n. 15

63. Mer 02/04/2025 ora n. 5 in S3B come Tutor del corso CyberSec lez. n. 16

64. Mer 02/04/2025 ora n. 6 in S4B come Tutor del corso CyberSec lez. n. 18

65. Ven 04/04/2025 ora n. 4 in S4B come Tutor del corso CyberSec lez. n. 19

66. Ven 04/04/2025 ora n. 6 in S3B come Tutor del corso CyberSec lez. n. 17

67. Mer 09/04/2025 ora n. 5 in S3B come Tutor del corso CyberSec lez. n. 18

68. Mer 09/04/2025 ora n. 6 in S4B come Tutor del corso CyberSec lez. n. 20

69. Ven 11/04/2025 ora n. 6 in S3B come Tutor del corso CyberSec lez. n. 19

70. Mer 16/04/2025 ora n. 5 in S3B come Tutor del corso CyberSec lez. n. 20

## Taccari

1. Mer 29/01/2025 ora n. 5 in S3B come Formatore del corso CyberSec lez. n. 1

2. Mer 29/01/2025 ora n. 6 in S4B come Formatore del corso CyberSec lez. n. 1

3. Ven 31/01/2025 ora n. 4 in S4B come Formatore del corso CyberSec lez. n. 2

4. Ven 31/01/2025 ora n. 6 in S3B come Formatore del corso CyberSec lez. n. 2

5. Mar 04/02/2025 ora n. 2 in S4G come Formatore del corso CryptoWithBlocks lez. n. 1

6. Mer 05/02/2025 ora n. 1 in S1B come Formatore del corso CryptoWithBlocks lez. n. 1

7. Mer 05/02/2025 ora n. 5 in S3B come Formatore del corso CyberSec lez. n. 3

8. Mer 05/02/2025 ora n. 6 in S4B come Formatore del corso CyberSec lez. n. 3

9. Ven 07/02/2025 ora n. 4 in S4B come Formatore del corso CyberSec lez. n. 4

10. Ven 07/02/2025 ora n. 6 in S3B come Formatore del corso CyberSec lez. n. 4

11. Sab 08/02/2025 ora n. 4 in S1B come Formatore del corso CryptoWithBlocks lez. n. 2

12. Mar 11/02/2025 ora n. 2 in S4G come Formatore del corso CryptoWithBlocks lez. n. 2

13. Mer 12/02/2025 ora n. 1 in S1B come Formatore del corso CryptoWithBlocks lez. n. 3

14. Mer 12/02/2025 ora n. 5 in S3B come Formatore del corso CyberSec lez. n. 5

15. Mer 12/02/2025 ora n. 6 in S4B come Formatore del corso CyberSec lez. n. 5

16. Sab 15/02/2025 ora n. 4 in S1B come Formatore del corso CryptoWithBlocks lez. n. 4

17. Mar 18/02/2025 ora n. 2 in S4G come Formatore del corso CryptoWithBlocks lez. n. 3

18. Mer 19/02/2025 ora n. 1 in S1B come Formatore del corso CryptoWithBlocks lez. n. 5

19. Mer 19/02/2025 ora n. 5 in S3B come Formatore del corso CyberSec lez. n. 6

20. Mer 19/02/2025 ora n. 6 in S4B come Formatore del corso CyberSec lez. n. 6

21. Ven 21/02/2025 ora n. 4 in S4B come Formatore del corso CyberSec lez. n. 7

22. Ven 21/02/2025 ora n. 6 in S3B come Formatore del corso CyberSec lez. n. 7

23. Sab 22/02/2025 ora n. 4 in S1B come Formatore del corso CryptoWithBlocks lez. n. 6

24. Mar 25/02/2025 ora n. 2 in S4G come Formatore del corso CryptoWithBlocks lez. n. 4

25. Mer 26/02/2025 ora n. 1 in S1B come Formatore del corso CryptoWithBlocks lez. n. 7

26. Mer 26/02/2025 ora n. 6 in S4B come Formatore del corso CyberSec lez. n. 8

27. Ven 28/02/2025 ora n. 4 in S4B come Formatore del corso CyberSec lez. n. 9

28. Sab 01/03/2025 ora n. 4 in S1B come Formatore del corso CryptoWithBlocks lez. n. 8

29. Mar 04/03/2025 ora n. 2 in S4G come Formatore del corso CryptoWithBlocks lez. n. 5

30. Mer 05/03/2025 ora n. 1 in S1B come Formatore del corso CryptoWithBlocks lez. n. 9

31. Mer 05/03/2025 ora n. 5 in S3B come Formatore del corso CyberSec lez. n. 8

32. Mer 05/03/2025 ora n. 6 in S4B come Formatore del corso CyberSec lez. n. 10

33. Ven 07/03/2025 ora n. 4 in S4B come Formatore del corso CyberSec lez. n. 11

34. Ven 07/03/2025 ora n. 6 in S3B come Formatore del corso CyberSec lez. n. 9

35. Sab 08/03/2025 ora n. 4 in S1B come Formatore del corso CryptoWithBlocks lez. n. 10

36. Mar 11/03/2025 ora n. 2 in S4G come Formatore del corso CryptoWithBlocks lez. n. 6

37. Mer 12/03/2025 ora n. 5 in S3B come Formatore del corso CyberSec lez. n. 10

38. Mer 12/03/2025 ora n. 6 in S4B come Formatore del corso CyberSec lez. n. 12

39. Ven 14/03/2025 ora n. 4 in S4B come Formatore del corso CyberSec lez. n. 13

40. Ven 14/03/2025 ora n. 6 in S3B come Formatore del corso CyberSec lez. n. 11

41. Mar 18/03/2025 ora n. 2 in S4G come Formatore del corso CryptoWithBlocks lez. n. 7

42. Mer 19/03/2025 ora n. 5 in S3B come Formatore del corso CyberSec lez. n. 12

43. Mer 19/03/2025 ora n. 6 in S4B come Formatore del corso CyberSec lez. n. 14

44. Ven 21/03/2025 ora n. 4 in S4B come Formatore del corso CyberSec lez. n. 15

45. Ven 21/03/2025 ora n. 6 in S3B come Formatore del corso CyberSec lez. n. 13

46. Mar 25/03/2025 ora n. 2 in S4G come Formatore del corso CryptoWithBlocks lez. n. 8

47. Mer 26/03/2025 ora n. 5 in S3B come Formatore del corso CyberSec lez. n. 14

48. Mer 26/03/2025 ora n. 6 in S4B come Formatore del corso CyberSec lez. n. 16

49. Ven 28/03/2025 ora n. 4 in S4B come Formatore del corso CyberSec lez. n. 17

50. Ven 28/03/2025 ora n. 6 in S3B come Formatore del corso CyberSec lez. n. 15

51. Mar 01/04/2025 ora n. 2 in S4G come Formatore del corso CryptoWithBlocks lez. n. 9

52. Mer 02/04/2025 ora n. 5 in S3B come Formatore del corso CyberSec lez. n. 16

53. Mer 02/04/2025 ora n. 6 in S4B come Formatore del corso CyberSec lez. n. 18

54. Ven 04/04/2025 ora n. 4 in S4B come Formatore del corso CyberSec lez. n. 19

55. Ven 04/04/2025 ora n. 6 in S3B come Formatore del corso CyberSec lez. n. 17

56. Mar 08/04/2025 ora n. 2 in S4G come Formatore del corso CryptoWithBlocks lez. n. 10

57. Mer 09/04/2025 ora n. 5 in S3B come Formatore del corso CyberSec lez. n. 18

58. Mer 09/04/2025 ora n. 6 in S4B come Formatore del corso CyberSec lez. n. 20

59. Ven 11/04/2025 ora n. 6 in S3B come Formatore del corso CyberSec lez. n. 19

60. Mer 16/04/2025 ora n. 5 in S3B come Formatore del corso CyberSec lez. n. 20

